In [13]:

import os

In [14]:
%pwd

'c:\\Users\\USER\\Documents\\GitHub\\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC'

In [3]:
os.chdir('../')

In [38]:
%pwd

'c:\\Users\\USER\\Documents\\GitHub\\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC'

In [39]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    target_column: str

In [40]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            target_column=schema.name
        )

        return data_transformation_config

In [42]:
import os
from mlFlowProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [53]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
    def preprocess_data(self, data):
        X = data.drop(columns=self.config.target_column)
        y = data[self.config.target_column]

        # Map the 'Gender' column
        mapping = {'Male': 0, 'Female': 1}
        X['Gender'] = X['Gender'].map(mapping)

        num_cols = X.select_dtypes(include=np.number).columns.to_list()
        cat_cols = X.select_dtypes(exclude=np.number).columns.to_list()

        num_pipeline = Pipeline(steps=[
            ('scaler', MinMaxScaler())
        ])

        cat_pipeline = Pipeline(steps=[
            ('one_hot_enc', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ])

        col_trans = ColumnTransformer(transformers=[
            ('num_pipeline', num_pipeline, num_cols),
            ('cat_pipeline', cat_pipeline, cat_cols),
        ], remainder='drop', n_jobs=-1)

        X_preprocessed = col_trans.fit_transform(X)

        # Get the list of output feature names from the column transformer
        feature_names = col_trans.get_feature_names_out()

        # Create a new DataFrame from the numpy array and assign the column names to it
        X_preprocessed = pd.DataFrame(X_preprocessed, columns=feature_names)

        return X_preprocessed, y

    
    def train_test_splitting(self):
        data = pd.read_csv(self.config.data_path)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data, test_size=0.25)

        # Preprocess the train and test data
        X_train, y_train = self.preprocess_data(train)
        X_test, y_test = self.preprocess_data(test)

        # Save the processed data
        train_processed = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)
        test_processed = pd.concat([X_test, y_test.reset_index(drop=True)], axis=1)

        train_processed.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test_processed.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Data split into training and test sets")
        logger.info(f"Train shape: {train.shape}")
        logger.info(f"Test shape: {test.shape}")

        print(f"Train shape: {train.shape}")
        print(f"Test shape: {test.shape}")

In [54]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_splitting()
except Exception as e:
    raise e

[2024-05-25 12:38:12,912: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-25 12:38:12,914: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-25 12:38:12,920: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-25 12:38:12,922: INFO: common: created directory at: artifacts]
[2024-05-25 12:38:12,923: INFO: common: created directory at: artifacts/data_transformation]
[2024-05-25 12:38:36,628: INFO: 2955539026: Data split into training and test sets]
[2024-05-25 12:38:36,630: INFO: 2955539026: Train shape: (7501, 14)]
[2024-05-25 12:38:36,631: INFO: 2955539026: Test shape: (2501, 14)]
Train shape: (7501, 14)
Test shape: (2501, 14)
